<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/master/2%20Advanced/340IsoMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 機械学習の例
### 教師なし学習の例
0〜4の5種類の文字を、ヒントなしに機械的に分類させる。それぞれの手書き文字の画像は、8x8=64ピクセルの濃淡でできているので、64次元のベクトルとみなせる。これを、多様体学習と呼ばれる手法を用い、2次元に落としこむ。コンピュータはこのデータが5種類の数字でできていることは知らない。64次元空間で近い点(似た画像)は、2次元空間でも近くなるように、射影される。(実際の計算時間は瞬時)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

# 画像データの読みこみ
digits = load_digits(n_class=5)

# 画像データの表示
fig, ax = plt.subplots(8, 8, figsize=(6, 6))
for i, axi in enumerate(ax.flat):
    axi.imshow(digits.images[i], cmap='binary')
    axi.set(xticks=[], yticks=[])

In [ ]:
# データの構造を見ておく。
digits, digits.images.shape

プログラムをざっと見てわかるように、パラメータとして、何種類の文字が含まれているかは教えていないし、個々の文字の正解(教師データ)も与えていない。しかし、IsoMapは自発的に文字の類似性を「発見」し、クラスターに分ける。

In [ ]:
# IsoMap (多様体学習の一手法； 次元を削減し、データの構造を明らかにする)
from sklearn.manifold import Isomap
iso = Isomap(n_components=2)
projection = iso.fit_transform(digits.data)

# 結果の描画
plt.scatter(projection[:, 0], projection[:, 1], lw=0.1,
           c='black')

それぞれの画像がどの数字を示しているかという情報もデータに含まれているが、コンピュータはそれを使わずにマッピングを行った。


どこに境界線を引くべきかは、Kmeansを使えば決められるだろう。ただし、その場合、5つのグループしか見付けられないかもしれない。

上の結果に対し、文字の種類ごとに異なる色を彩色してやると、6種類の文字が異なる島に分かれている=識別されていることがわかる。



In [ ]:
# 文字の種類ごとに異なる色を付けて描画
plt.scatter(projection[:, 0], projection[:, 1], lw=0.1,
           c=digits.target, cmap=plt.cm.get_cmap('jet', 5))
plt.colorbar(ticks=range(5), label='digit value')
plt.clim(-0.5, 4.5)

試しに、以前NNの学習に利用した、28x28の文字データでもやってみよう。

In [ ]:
import matplotlib.pyplot as plt
# from sklearn.datasets import load_digits
from keras.datasets import mnist

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 画像データの表示
fig, ax = plt.subplots(8, 8, figsize=(6, 6))
for i, axi in enumerate(ax.flat):
    axi.imshow(x_train[i], cmap='binary')
    axi.set(xticks=[], yticks=[])

In [ ]:
# データの構造は?
x_train.shape

データセットが大きすぎるので、5000個だけ使う。

In [ ]:
# 5000個の576次元ベクトル
vectors = x_train.reshape(-1, 28*28)[:5000]
# 5000個の答 (文字)
answers = y_train[:5000]

In [ ]:
# IsoMap (多様体学習の一手法； 次元を削減し、データの構造を明らかにする)
from sklearn.manifold import Isomap

# 2次元では重なってよくわからないので3次元にする。
iso = Isomap(n_components=3)
projection = iso.fit_transform(vectors)


pyplotで3次元散布図にし、それに本来の文字の分類を色としてのせる。

In [ ]:
import plotly.graph_objs as go
import plotly.express as px

trace=dict(type='scatter3d',
           x= projection[:,0],
           y= projection[:,1],
           z= projection[:,2],
           mode='markers',
           marker=dict(color=answers,
                       colorscale='rainbow',
                       size=3)
          )
fig = go.Figure(data=trace)
fig.update_layout(scene = dict(
                    xaxis_title='R',
                    yaxis_title='G',
                    zaxis_title='B'))
fig.show()

さすがに、画像が大きくなってくると、3次元への射影でも十分に分離できなくなってくる。

ひとたびこのようなテリトリーの地図が得られると、未知の手書き文字を同じようにこの地図にのせることで、それがどの文字に一番近いかを「認識」できるようになる。

この例では、教師データは与えず、検証にしか利用していない。先験的な知識なしに、プログラムが文字をその形だけで分類できることを示している。

コンピュータにいろんな国の言葉を聞かせ、それがどの言語かを教えてやれば、言語の特徴を学習して、それがどの言語かを言いあてることができるようになる。(教師あり学習)

それに対し、コンピュータに何も教えないで、ただ類似性だけで分類させる(教師なし学習)ことができれば、ある言語が別の言語と近いか遠いかを見分けられるようになるかもしれない。例えば、名古屋弁と岡山弁は実は似ていた、といった新しい発見をもたらすかもしれない。人間のまねをさせるだけなら教師あり学習が有用だが、分析を行う人にとっては教師なし学習のほうが魅力的に見える。

## 参考資料

* Pythonデータサイエンスハンドブック
* パターン認識と機械学習